## Building Docker Artifacts

In [ ]:
cd farmer

In [ ]:
!docker login -u username -p password

In [100]:
%%writefile config.py
gs_bucket_name="data-labeling-demo"
Bucket_uri="gs://data-labeling-demo"
version=1
store_artifacts=Bucket_uri + "/" + str(version)
data_path=Bucket_uri + "/" + "data/data_raw.csv"
processed_data=Bucket_uri + "/" + "processed/data_processed.csv"

Overwriting config.py


In [101]:
%%writefile Dockerfile
FROM tensorflow/tensorflow:2.2.0-gpu
ARG DEBIAN_FRONTEND=noninteractive
# Install apt dependencies
RUN apt-get update && apt-get install -y \
    git \
    gpg-agent \
    python3-cairocffi \
    protobuf-compiler \
    python3-pil \
    python3-lxml \
    python3-tk \
    wget
# Install gcloud and gsutil commands
# https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
RUN export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)" && \
    echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - && \
    apt-get update -y && apt-get install google-cloud-sdk -y
WORKDIR /pipeline
COPY ./ ./
RUN pip install -r requirements.txt
RUN pip install "dask[dataframe]" --upgrade
ENV TF_CPP_MIN_LOG_LEVEL 3

Overwriting Dockerfile


## Build Docker Image

In [102]:
!docker build -t doctorai/kubeflow-sdk-farmer .

Sending build context to Docker daemon  109.6kB
Step 1/9 : FROM tensorflow/tensorflow:2.2.0-gpu
 ---> f5ba7a196d56
Step 2/9 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> 9fc4229fc73e
Step 3/9 : RUN apt-get update && apt-get install -y     git     gpg-agent     python3-cairocffi     protobuf-compiler     python3-pil     python3-lxml     python3-tk     wget
 ---> Using cache
 ---> 73c89cdcd934
Step 4/9 : RUN export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)" &&     echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list &&     curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - &&     apt-get update -y && apt-get install google-cloud-sdk -y
 ---> Using cache
 ---> eb9e17807a13
Step 5/9 : WORKDIR /pipeline
 ---> Using cache
 ---> 62528170fddb
Step 6/9 : COPY ./ ./
 ---> 04e4e27fed5a
Step 7/9 : RUN pip install -r requirements.txt
 ---> Running in 33e90c3cc477
  Created wheel for wg

## Push Docker Image to Docker hub

In [103]:
!docker push doctorai/kubeflow-sdk-farmer

The push refers to repository [docker.io/doctorai/kubeflow-sdk-farmer]

8991fce1: Preparing 
61f632ae: Preparing 
e9a60d35: Preparing 
b37303ae: Preparing 
8935924b: Preparing 
ea521373: Preparing 
e55f84c6: Preparing 
b0f92c14: Preparing 
cf4cd527: Preparing 
c1f74e01: Preparing 
9e4b0fc9: Preparing 
e3b79e0a: Preparing 
e43735a0: Preparing 
3918ca41: Preparing 
768f66a4: Preparing 
d332a58a: Preparing 
f11cbf29: Preparing 
a4b22186: Preparing 
afb09dc3: Preparing 
b5a53aac: Preparing 
c8e5063e: Preparing 
1f632ae: Pushed   348.7MB/337.8MB2Alatest: digest: sha256:2bb906b2e53060b6ea0b6d8f6b3fb03a46fd029faa504a0184d3580115828750 size: 4938


## Install Kubeflow SDk

In [104]:
!pip install kfp

# Building Kubeflow Pipeline

In [105]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

## Define kubeflow pipeline Component

In [106]:
@kfp.dsl.component
def get_data():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='docker.io/doctorai/kubeflow-sdk-farmer',
        command=['python', 'get_data.py'],
        )
    return training_component

In [107]:
@kfp.dsl.component
def data_preprocessing():    
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='docker.io/doctorai/kubeflow-sdk-farmer',
        command=['python', 'process_data.py'],
        )
    return training_component

In [108]:
@kfp.dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='docker.io/doctorai/kubeflow-sdk-farmer',
        command=['python', 'train.py'],
        file_outputs={'mlpipeline-ui-metadata':'/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    
    return training_component

In [109]:
# Let see output of component configuration
debug = False 
if debug :
    training_component_vis = tensorflow_object_detection_api_training_component()
    print(training_component_vis)

## Final Kubeflow pipeline Definition

In [110]:
@kfp.dsl.pipeline(
  name="Modeling Swiss farmer's attitudes about",
  description="Modeling Swiss farmer's attitudes about"
)
def farmer():
    download_data = get_data()
    download_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    data_processing = data_preprocessing().after(download_data)
    data_processing.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train = training().after(data_processing)
    train.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [111]:
# Let see output of pipeline configuration
debug = False 
if debug :
    training_pipeline_output = farmer()
    print(training_component_output)

## Compile Kubeflow Pipeline 
* It will Generate .zip file inside this contain YAMl file which contain the configuration of kubeflow pipeline

In [112]:
kfp.compiler.Compiler().compile(farmer, 'farmer-kubeflow-sdk-demo.zip')

## Connect to deployed kubeflow pipeline Endpoint (GCP)

In [113]:
# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
client = kfp.Client("118d949fbd12e5f7-dot-us-central2.pipelines.googleusercontent.com")

### Create Experiment 

In [114]:
EXPERIMENT_NAME = 'farmer-1'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

### Deploy pipeline to kubeflow pipeline Endpoint

In [115]:
run = client.run_pipeline(experiment.id, 'farmer-run-1', 'farmer-kubeflow-sdk-demo.zip')